In [56]:
import pandas as pd

In [82]:
path = r"C:\Users\e_koffie\Documents\IAI_Project\IAI_PROJECT\output_VF\region\eloignement\terrain\securite_terrain_eloignement.xlsx"

In [83]:
df = pd.read_excel(path)

In [59]:
df.head()

,NomReg,eloignement_090010001,eloignement_090010002,eloignement_090010003,eloignement_090010004,eloignement_global_09001,eloignement_090020001,eloignement_090020002,eloignement_090020003,eloignement_global_09002,eloignement_090030001,eloignement_090030002,eloignement_090030003,eloignement_global_09003,TotalPopIL,CodReg,NomReg.1
0,ABIDJAN,0.259512,0.006559,0.003906,0.006632,0.069152,0.005795,0.008505,0.012515,0.008938,0.012515,0.095729,0.173209,0.093818,6.285900e+06,1,ABIDJAN
1,AGNEBY-TIASSA,0.227159,0.197858,0.240658,0.162918,0.207148,0.154902,0.262813,0.304261,0.240659,0.304261,0.048480,0.107409,0.153383,7.178046e+05,16,AGNEBY-TIASSA
2,BAFING,0.726075,0.723840,0.923750,0.874540,0.812051,0.443788,0.653758,0.723793,0.607113,0.723793,0.911556,0.621891,0.752413,2.335946e+05,19,BAFING
3,BAGOUE,0.379022,0.330644,0.127014,0.820031,0.414178,0.175544,0.340184,0.104937,0.206888,0.104937,0.636245,0.307330,0.349504,4.921749e+05,20,BAGOUE
4,BELIER,0.253327,0.461560,0.163877,0.345109,0.305968,0.223979,0.274044,0.407889,0.301971,0.407889,0.312748,0.383071,0.367903,3.889655e+05,21,BELIER


In [60]:
def compute_weighted_means(df, weight_col='TotalPopIL'):
    df_clean = df.copy()

    # Convert weight column from string with commas to float
    df_clean[weight_col] = df_clean[weight_col].astype(str).str.replace(',', '.').astype(float)

    # Identify eloignement columns
    eloignement_cols = [col for col in df_clean.columns if col.startswith('eloignement_')]

    # Convert eloignement columns from string with commas to float
    for col in eloignement_cols:
        df_clean[col] = df_clean[col].astype(str).str.replace(',', '.').astype(float)

    # Compute weighted mean for each eloignement column
    weighted_means = {
        col: (df_clean[col] * df_clean[weight_col]).sum() / df_clean[weight_col].sum()
        for col in eloignement_cols
    }

    # Compute national weighted mean (excluding 'global' columns)
    non_global_cols = [col for col in eloignement_cols if 'global' not in col]
    national_mean = sum(
        (df_clean[col] * df_clean[weight_col]).sum() for col in non_global_cols
    ) / (df_clean[weight_col].sum() * len(non_global_cols))

    # Add to results
    weighted_means['eloignement_national'] = national_mean

    # Return as a single-row DataFrame
    return pd.DataFrame([weighted_means])

In [61]:
path_dec = r"C:\Users\e_koffie\Documents\IAI_Project\IAI_PROJECT\Data\Infrastructures\dictionnaires_secteurs.xlsx"

In [62]:
def get_sector_info(df, code):
    """
    Given a hierarchical code, return a dictionary of corresponding elements:
    - If code has 2 digits: return {"Nom du secteur": ...}
    - If code has 5 digits: return {"Nom du secteur": ..., "Sous secteur": ...}
    - If code has 9 digits: return {"Nom du secteur": ..., "Sous secteur": ..., "Description": ...}
    """

    code = str(code)

    if len(code) == 2:
        match = df[df['code'] == code]
        if not match.empty:
            return {
                "Nom du secteur": match['Nom du secteur'].iloc[0]
            }

    elif len(code) == 5:
        code_part = code[:2]
        id_secteur = code[2:5]
        match = df[(df['code'] == code_part) & (df['id_secteur'] == id_secteur)]
        if not match.empty:
            return {
                "Nom du secteur": match['Nom du secteur'].iloc[0],
                "Sous secteur": match['Sous secteur'].iloc[0]
            }

    elif len(code) == 9:
        code_part = code[:2]
        id_secteur = code[2:5]
        id_hiera = code[5:9]
        match = df[
            (df['code'] == code_part) &
            (df['id_secteur'] == id_secteur) &
            (df['id_Hiera'] == id_hiera)
        ]
        if not match.empty:
            return {
                "Nom du secteur": match['Nom du secteur'].iloc[0],
                "Sous secteur": match['Sous secteur'].iloc[0],
                "Description": match['Description'].iloc[0]
            }

    return None  # If no match or invalid code


In [63]:
get_sector_info(df_dec, "060030004")

In [64]:
import numpy as np

def extract_subsectors(df_weighted, sector_df):
    rows = []

    for col in df_weighted.columns:
        if not col.startswith('eloignement_'):
            continue

        code_part = col.replace('eloignement_', '')
        value = df_weighted[col].values[0]

        # Cas spécial : national
        if code_part == 'national':
            rows.append({
                "Nom variable": col,
                "Code": "national",
                "Nom du secteur": np.nan,
                "Nom du sous secteur": np.nan,
                "Description": np.nan,
                "Eloignement": round(value, 9)
            })
            continue

        # Cas global : comme une ligne normale, mais sans Description
        if col.startswith('eloignement_global_'):
            code = code_part  # ex: 06001
            info = get_sector_info(sector_df, code)
            rows.append({
                "Nom variable": col,
                "Code": code,
                "Nom du secteur": info.get("Nom du secteur", "") if info else np.nan,
                "Nom du sous secteur": info.get("Sous secteur", "") if info else np.nan,
                "Description": np.nan,
                "Eloignement": round(value, 9)
            })
            continue

        # Cas standard : code complet (9 chiffres)
        info = get_sector_info(sector_df, code_part)
        rows.append({
            "Nom variable": col,
            "Code": code_part,
            "Nom du secteur": info.get("Nom du secteur", "") if info else np.nan,
            "Nom du sous secteur": info.get("Sous secteur", "") if info else np.nan,
            "Description": info.get("Description", "") if info and "Description" in info else np.nan,
            "Eloignement": round(value, 9)
        })

    return pd.DataFrame(rows)



In [65]:
df_dec = pd.read_excel(path_dec, dtype={"code": str, "id_secteur": str, "id_Hiera": str})

In [67]:
df_d = extract_subsectors(df_c, df_dec)

In [68]:
df_d

,Nom variable,Code,Nom du secteur,Nom du sous secteur,Description,Eloignement
0,eloignement_090010001,090010001,SANTE,Services de soins privé,Dépots de médicaments,0.312291
1,eloignement_090010002,090010002,SANTE,Services de soins privé,Pharmacie,0.256318
2,eloignement_090010003,090010003,SANTE,Services de soins privé,Centre de santé/clinique chinoise/tradipratici...,0.284095
3,eloignement_090010004,090010004,SANTE,Services de soins privé,Clinique / Polyclinique,0.265147
4,eloignement_global_09001,global_09001,NaN,NaN,NaN,0.279463
5,eloignement_090020001,090020001,SANTE,Services de soins public,"1er niveau de référence (CSR, CSU …)",0.190992
6,eloignement_090020002,090020002,SANTE,Services de soins public,"2ieme niveau de référence (CHR, HG ...)",0.294736
7,eloignement_090020003,090020003,SANTE,Services de soins public,"3ieme niveau de référence (CHU, INPH, SAMU …)",0.316858
8,eloignement_global_09002,global_09002,NaN,NaN,NaN,0.267529
9,eloignement_090030001,090030001,SANTE,Administration décentralisée,District Sanitaire,0.316858


In [66]:
df_c = compute_weighted_means(df)

In [16]:
df_c

,eloignement_100010001,eloignement_global_10001,eloignement_100020001,eloignement_100020002,eloignement_100020003,eloignement_100020004,eloignement_100020005,eloignement_global_10002,eloignement_100030001,eloignement_100030002,...,eloignement_100040001,eloignement_global_10004,eloignement_100050001,eloignement_100050002,eloignement_100050003,eloignement_100050004,eloignement_global_10005,eloignement_global_10006,eloignement_global_10007,eloignement_national
0,0.322888,0.322888,0.333333,0.32565,0.30655,0.329022,0.331822,0.325275,0.333333,0.30935,...,0.329632,0.329632,0.333207,0.333333,0.333333,0.330517,0.332598,0.0,0.0,0.327031


In [69]:
path_save = r"C:\Users\e_koffie\Documents\IAI_Project\sante___terrain_eloignement_weighted_means.xlsx"

In [70]:
df_d.to_excel(path_save, index=False)

In [71]:
import numpy as np
import pandas as pd

def extract_subsectors_by_region(df_weighted, sector_df):
    all_rows = []

    for _, row in df_weighted.iterrows():
        nom_region = row["NomReg"]
        
        for col in df_weighted.columns:
            if not col.startswith("eloignement_"):
                continue

            code_part = col.replace("eloignement_", "")
            value = row[col]

            # Cas spécial : national
            if code_part == 'national':
                all_rows.append({
                    "Région": nom_region,
                    "Nom variable": col,
                    "Code": "national",
                    "Nom du secteur": np.nan,
                    "Nom du sous secteur": np.nan,
                    "Description": np.nan,
                    "Eloignement": round(value, 9)
                })
                continue

            # Cas global
            if col.startswith('eloignement_global_'):
                code = code_part
                info = get_sector_info(sector_df, code)
                all_rows.append({
                    "Région": nom_region,
                    "Nom variable": col,
                    "Code": code,
                    "Nom du secteur": info.get("Nom du secteur", "") if info else np.nan,
                    "Nom du sous secteur": info.get("Sous secteur", "") if info else np.nan,
                    "Description": np.nan,
                    "Eloignement": round(value, 9)
                })
                continue

            # Cas standard
            info = get_sector_info(sector_df, code_part)
            all_rows.append({
                "Région": nom_region,
                "Nom variable": col,
                "Code": code_part,
                "Nom du secteur": info.get("Nom du secteur", "") if info else np.nan,
                "Nom du sous secteur": info.get("Sous secteur", "") if info else np.nan,
                "Description": info.get("Description", "") if info and "Description" in info else np.nan,
                "Eloignement": round(value, 9)
            })

    return pd.DataFrame(all_rows)


In [84]:
reg = extract_subsectors_by_region(df, df_dec)

In [85]:
path_reg = r"C:\Users\e_koffie\Documents\IAI_Project\securite_region__terrain_eloign.xlsx"

In [86]:
reg.to_excel(path_reg, index=False)